In [2]:
import csv
import requests
import time
import os
from datetime import datetime

In [3]:
def check_virustotal(api_key, ioc):
    """
    Check an indicator of compromise (IOC) against VirusTotal API
    
    Args:
        api_key (str): Your VirusTotal API key
        ioc (str): The indicator to check (hash, URL, domain, IP)
        
    Returns:
        dict: Results including detection stats and error message if any
    """
    base_url = "https://www.virustotal.com/api/v3/"
    headers = {"x-apikey": api_key}
    result = {
        "malicious": 0,
        "total": 0,
        "permalink": "",
        "error": ""
    }
    
    # Determine IOC type and create appropriate endpoint
    if all(c in '0123456789abcdefABCDEF' for c in ioc) and len(ioc) in [32, 40, 64]:
        # MD5, SHA1, or SHA256 hash
        endpoint = f"files/{ioc}"
        result["permalink"] = f"https://www.virustotal.com/gui/file/{ioc}"
    elif len(ioc.split('.')) == 4 and all(o.isdigit() and int(o) >= 0 and int(o) <= 255 for o in ioc.split('.')):
        # IP address
        endpoint = f"ip_addresses/{ioc}"
        result["permalink"] = f"https://www.virustotal.com/gui/ip-address/{ioc}"
    elif '.' in ioc and all(part.isalnum() for part in ioc.split('.')):
        # Domain
        endpoint = f"domains/{ioc}"
        result["permalink"] = f"https://www.virustotal.com/gui/domain/{ioc}"
    else:
        # URL or other
        import base64
        url_id = base64.urlsafe_b64encode(ioc.encode()).decode().strip("=")
        endpoint = f"urls/{url_id}"
        result["permalink"] = f"https://www.virustotal.com/gui/url/{url_id}"
    
    try:
        response = requests.get(base_url + endpoint, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            stats = data.get("data", {}).get("attributes", {}).get("last_analysis_stats", {})
            result["malicious"] = stats.get("malicious", 0) + stats.get("suspicious", 0)
            result["total"] = sum(stats.values()) if stats else 0
        elif response.status_code == 404:
            result["error"] = "IOC not found in VirusTotal database"
        elif response.status_code == 429:
            result["error"] = "API rate limit exceeded"
        else:
            result["error"] = f"API error: {response.status_code}"
            
    except Exception as e:
        result["error"] = f"Request error: {str(e)}"
        
    return result

In [4]:
# Function to process the CSV
def process_iocs(api_key, csv_path, ioc_column="ioc"):
    # Create output filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"vt_results_{timestamp}.csv"
    
    results = []
    
    # Read the CSV file
    with open(csv_path, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        
        if ioc_column not in reader.fieldnames:
            return f"Error: Column '{ioc_column}' not found in CSV. Available columns: {', '.join(reader.fieldnames)}"
        
        # Process each row
        for i, row in enumerate(reader):
            ioc = row[ioc_column].strip()
            if not ioc:
                continue
                
            print(f"Checking IOC {i+1}: {ioc}")
            
            # Check the IOC against VirusTotal
            vt_result = check_virustotal(api_key, ioc)
            
            # Add results to the row
            result_row = row.copy()
            result_row["vt_malicious"] = vt_result["malicious"]
            result_row["vt_total"] = vt_result["total"]
            result_row["vt_detection_rate"] = f"{(vt_result['malicious'] / vt_result['total'] * 100):.2f}%" if vt_result["total"] > 0 else "N/A"
            result_row["vt_permalink"] = vt_result["permalink"]
            result_row["vt_error"] = vt_result["error"]
            
            results.append(result_row)
            
            # Respect VirusTotal API rate limits (4 requests/minute for public API)
            if vt_result["error"] != "API rate limit exceeded":
                time.sleep(15)  # 15 seconds between requests to stay under limit
            else:
                print("Rate limit reached. Waiting 60 seconds...")
                time.sleep(60)  # Wait longer if rate limited
    
    # Write results to output CSV
    output_fieldnames = reader.fieldnames + ["vt_malicious", "vt_total", "vt_detection_rate", "vt_permalink", "vt_error"]
    with open(output_file, 'w', newline='') as output_csv:
        writer = csv.DictWriter(output_csv, fieldnames=output_fieldnames)
        writer.writeheader()
        writer.writerows(results)
        
    return f"Processing complete. Results saved to {output_file}"

In [5]:
# Set your parameters here
api_key = "f611a659631959cbb48e44690597093b32d525167ba58c773ef72a8e4a307eed"  # Replace with your actual API key
csv_path = "daily_iocs.csv"           # Replace with your CSV file path
ioc_column = "ioc"                   # Replace with your IOC column name if different

# Process the IOCs
result_message = process_iocs(api_key, csv_path, ioc_column)
print(result_message)

Checking IOC 1: hxxps://hilarious-trifle-d9182e.netlify.app/lootsubmit.ps1
Checking IOC 2: 44b7eebf7a26d466f9c7ad4ddb058503f7066aded180ab6d5162197c47780293
Checking IOC 3: 3d2cbef9be0c48c61a18f0e1dc78501ddabfd7a7663b21c4fcc9c39d48708e91
Processing complete. Results saved to vt_results_20250423_201931.csv
